In [1]:
%load_ext autoreload
%autoreload 2

import os
import tensorflow as tf

# GPU 메모리 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [8]:
from data_split import run_data_split, run_create_validation_csv
from model import run_training
from visualize import plot_loss_curve, plot_accuracy_curve, evaluate_model_visual

In [9]:
CONFIG = {
    "BASE_DIR": r'F:\ML\dataset',
    "EPOCHS": 20,
    "BATCH_SIZE": 32,
    "SAMPLES_PER_EPOCH_PER_GENRE": 10000,
    "MODEL_SAVE_NAME": "game_genre_cnn_v1.h5",
    "DO_DATA_SPLIT": False  # 최초 실행 시에만 True로 설정
}

In [10]:
# 1. 데이터셋 준비 (필요 시)
if CONFIG["DO_DATA_SPLIT"]:
    run_data_split()
    run_create_validation_csv()

# 2. 모델 학습
# model.py의 run_training 함수를 호출합니다.
model, history, val_dataset = run_training(
    epochs=CONFIG["EPOCHS"],
    batch_size=CONFIG["BATCH_SIZE"],
    samples_per_genre=CONFIG["SAMPLES_PER_EPOCH_PER_GENRE"],
    base_dir=CONFIG["BASE_DIR"]
)

🚚 데이터 매핑 로딩 중: training_tag_vectors.csv
🚚 데이터 매핑 로딩 중: validation_tag_vectors.csv

🚀 학습 시작 (총 20 Epochs, 에폭당 장르별 1000개 샘플링)

[Epoch 1/20]
194/344 [===============>..............] - ETA: 14s - loss: 0.5615 - accuracy: 0.2792 - exact_match_ratio: 0.0150 - micro_f1_score: 0.3407 - jaccard_similarity: 0.2197 - precision: 0.5248 - recall: 0.2573          

KeyboardInterrupt: 

In [ ]:
# 학습 곡선 출력 (loss, accuracy 등)
plot_loss_curve(model_history)
plot_accuracy_curve(model_history)
    
# 상세 성능 분석 (장르별 컨퓨전 매트릭스)
evaluate_model_visual(model, val_dataset)

# 모델 저장
save_path = os.path.join(CONFIG["BASE_DIR"], CONFIG["MODEL_SAVE_NAME"])
model.save(save_path)
print(f"✅ 모델 저장 완료: {save_path}")